<font color=blue>**SEVER: Learning in the Presence of Outliers**</font>
===========================
---
## Gabe Mancino-Ball
### CSCI 6971 - R.P.I. - Spring 2019

Link to original paper [here](https://arxiv.org/pdf/1803.02815.pdf)

Authors: I. Diakonikolas, G. Kamath, D. Kane, J. Li, J. Steinhardt, A.  Stewart

Published at **NeurIPS 2018 Workshop on Security in Machine Learning**

# Problem Setting
---

**Stochastic Optimization**

$$\min_{w\in\mathcal{H}}\bar{f}(w):=\mathbf{E}_{f\sim p^*}[f(w)]$$

where $p^*$ is a distribution of functions $f:\mathcal{H}\to\mathbf{R}$ and our *goal* is to find $w^*\in\mathcal{H}$

# Caveat
---

>We have structured **outliers** planted in our data

This is reasonable in many settings

# Assumptions
---

- $g(\cdot)$ is **linear** combination of features, i.e. $g(w)=w^\top x$
- $f(\cdot):=l(g(\cdot))$, i.e. $f$ is our **loss** function

#### This is Empirical Risk Minimization!

# Loss Functions
---

#### $l_2$-Loss
$$f(w)=\frac{1}{2}(w^\top x-y)^2$$
for regression

#### Hinge Loss

$$f(w)=\max(0,1-y(w^\top x))$$
for classification


# Proposed Solution
---


<img src='severmap.png' width='90%'>

# Algorithm Motivation
---
- Fit a model on original training data containing $S$ points to learn $\hat{w}$
- Compute the gradients of $f_{i\in S}(\hat{w})$
- Compute the **average** gradient $\hat{\nabla}:=\frac{1}{\lvert S\rvert}\sum_{i\in S}\nabla f_i(\hat{w})$
- Compare $\hat{\nabla}$ to $\nabla f_{i\in S}(\hat{w})$ by forming a matrix $G:=[\nabla f_i(\hat{w})-\hat{\nabla}]_{i\in S}$
- Remove points that satisfy a certain criteria

# Removal of Points and Outlier Scores
---
- Compute the SVD of $G$ to reveal space of gradients
- If this space is corrupted, some singular values will be much larger than others
- Let $v$ be the top right singular vector
- If $i^{th}$ **data point** is corrupted, we expect $\nabla f_i(\hat{w})-\hat{\nabla}$ to be large
- Compute $(\nabla f_i(\hat{w})-\hat{\nabla})^\top v$
- If this is "large" this implies that these vectors point in the same direction so remove this data point

# SEVER Algorithm
---

**Input:** Sample functions $f_1,\dots,f_n:\mathcal{H}\to\mathbb{R}$, Learner $\mathcal{L}$, Parameter $\sigma>0.$

**Initialize:** $S=\{1,\dots,n\}$

Compute: $w=\mathcal{L}(\{f_i\}_{i\in S})$, $\hat{\nabla}:=\frac{1}{\lvert S\rvert}\sum_{i\in S}\nabla f_i(w)$, $G:=[\nabla f_i(w)-\hat{\nabla}]_{i\in S}$, $v$ from SVD of $G$

Compute *outlier scores* $\gamma_i=\big((\nabla f_i(w)-\hat{\nabla})^\top v\big)^2$

Filter $S$ 

# Filter Algorithm
---
Compute $\sum_i\gamma_i$ and if this is less than $c\sigma$ (for some $c>1$) then remove no points and **terminate the algorithm**

Otherwise draw $T\sim UNI(0,\max_i\gamma_i)$ and throw out points $j$ such that $\gamma_j<T$ and **repeat the algorithm**

# Theoretical Guarantees
---
We require that **at most** 50% of the data is corrupted

- Number of sample points is proportional to dimension of data
- `SEVER` terminates in at most $n$ iterations
- 9/10 of the time the returned $w^*$ is a critical point of $\mathbf{E}_{f\sim p^*}[f(w)]$

# Numerical Results
---

- Test on Ridge Regression Problem
- Use artificial data (generated from Gaussian distribution)
- Use song prediction data
- Use Factorized Regularized Conjugate Gradient to solve Ridge Regression Problem

# Attack Data
---
- If $(x,y)$ are original data, create malicious data as follows:
$$x_{bad}=\frac{1}{\alpha\cdot n_{bad}}y^\top x$$
$$y_{bad}=-\beta$$
- Causes Ridge Regression to output $w=0$

# Compared Methods
---
- `TheorySEVER` (Proposed)
- `PracSEVER` (Practical): remove top $p$ percent of outliers for a set number of iterations
- `GradCenter`: remove top $p$ percent of outliers$^*$ for a set number of iterations
- `CG`

$^*$ outliers are measured by $\lVert\nabla f_i(w)-\hat{\nabla}\rVert_2$

In [ ]:
# Import relevant packages

import numpy as np
import math
import time
import random as rnd
from numpy import linalg as la
import matplotlib.pyplot as plt
import pandas as pd

# For displaying images side by side
from IPython.display import HTML, display

In [ ]:
%%HTML
<style>
td {
  font-size: 25px
}
</style>

# Solvers for Ridge Regression Problem

In [ ]:
# Include solvers for ridge regression case

def BlackBoxA(A, X, gam):
    '''
    Compute (A^tA+gam*I)X

    INPUTS:
    -------
    A = feature matrix
    X = weight matrix
    gam = regularized parameter

    OUTPUTS:
    --------
    ax = (A^tA+gam*I)X
    '''

    n = A.shape[1]

    ax = np.dot(A, X)
    ax = np.dot(np.transpose(A), ax)
    ax = ax + np.dot(gam*np.eye(n), X)

    return ax


def FactorizedRegularizedCG(BlackBoxA, A, Y, gamma, maxIters, eps, x0):
    '''
    Conjugate Gradient for Solving Linear Ridge Regression

    INPUTS:
    -------
    BlackBoxA = function that computes ~A*x
    A = matrix of features (rows are features)
    Y = vector of targets
    gamma = regularization constant
    maxIters = maximum number of iterations
    eps = error tolerance
    x0 = initial guess for x

    OUTPUTS:
    --------
    X = feature weight matrix
    numIters = number of iterations ran as a vector
    timeInside = total time spent evaluating the function
    '''

    tstart = time.time()

    [n, d] = A.shape
    m = len(Y)

    # Conjugate Gradient Algorithm

    Y = np.dot(np.transpose(A), Y)

    # Initialization
    comp = 0
    ynorm = la.norm(Y)
    X = x0

    res = Y - BlackBoxA(A, X, gamma)
    p = res

    for i in range(maxIters):

        numIters = i + 1

        ap = BlackBoxA(A, p, gamma)

        # Compute alpha step size
        num = np.dot(np.transpose(res), res)
        den = np.dot(np.transpose(p), ap)
        alpha = num / den

        # Update iterate and residual
        X = X + alpha*p
        res = res - alpha*ap

        # Check termination criteria
        multnorm = la.norm(BlackBoxA(A, X, gamma) - Y, axis=0)
        check = multnorm - eps*ynorm
        sol = np.less_equal(check, comp).flatten()

        if np.sum(sol) == sol.shape[0]:
            # print('All targets have converged. We are on iterate:', numIters, '.')
            break

        # Update conjugate search directions
        bnum = np.dot(np.transpose(res), res)
        beta = bnum / num
        p = res + beta*p

    tend = time.time()
    timeInside = (tend - tstart)*1000

    return [X, numIters, timeInside]


def regGrad(x, y, w, lam):
    '''
    Gradient for Ridge Regression Function
    
    INPUTS:
    -------
    x = feature matrix (ith row is feature vector)
    y = target vector
    w = optimal parameter vector
    lam = regularization parameter
    
    OUTPUTS:
    -------
    grad = gradient evaluated at w where ith column is grad_i
    '''
    
    lhs = np.diag(np.dot(x, w) - y)
    grad = np.dot(lhs, x) + lam*w
    grad = np.transpose(grad)
    
    return grad

# Code for Generating Synthetic Data

In [ ]:
# Generate synthetic data as described in the paper

def genRan(n, t, l, w, tog, nbad, alpha, beta):
    '''
    Generate Random Samples
    
    INPUTS:
    -------
    n = number of training samples
    t = number of testing samples
    l = length of data vector
    w = optimal parameter of problem
    tog = toggle for whether we want data for regression or classification
    (1 for classification, anything else for regression)
    nbad = number of attack data
    alpha, beta = attack parameters
    
    OUTPUTS:
    -------
    x = feature data
    y = target data
    xt = test feature data
    yt = test target data
    '''
    
    # Toggle for classification
    if tog == 1:
    
        # Generate all data
        x = np.random.normal(loc=0, scale=1, size=(n+t, l))
        z = np.random.normal(loc=0, scale=1, size=n+t)
        y = np.dot(x, w) + z
    
        # Generate synthetic attack points
        if nbad > 0:
            badsamp = np.array(rnd.sample((range(n+t)), nbad))
            x[badsamp] = (1/(nbad*alpha))*np.dot(np.transpose(y[badsamp]), x[badsamp])
            y[badsamp] = -beta
        
        # Change to class
        y = np.sign(y)
        yt = np.sign(yt)
        
        # Split into testing and training data
        xt = x[n:n+t]
        yt = y[n:n+t]
        x = x[0:n]
        y = y[0:n]
    
    # Else only contaminate the training set for regression
    else:
    
        # Generate training data
        x = np.random.normal(loc=0, scale=1, size=(n, l))
        z = np.random.normal(loc=0, scale=1, size=n)
        y = np.dot(x, w) + z
     
        # Generate testing data
        xt = np.random.normal(loc=0, scale=1, size=(t, l))
        zt = np.random.normal(loc=0, scale=1, size=t)
        yt = np.dot(xt, w) + zt
    
        # Generate synthetic attack data on testing set
        if nbad > 0:
            badsamp_test = np.array(rnd.sample((range(n)), nbad))
            x[badsamp_test] = (1/(nbad*alpha))*np.dot(np.transpose(y[badsamp_test]), x[badsamp_test])
            y[badsamp_test] = -beta
        
    
    return [x, y, xt, yt]

# Code for Corrupting Data Sets

In [ ]:
def dataAttack(x, y, pb, alpha, beta):
    '''
    Corrupt a fraction of training data
    
    INPUTS:
    -------
    x = feature matrix
    y = target vector
    pb = percent of samples you want to corrupt
    alpha, beta = attack paramters
    
    OUTPUTS:
    --------
    xc = corrupted features
    yc = corrupted targets
    '''
    [n, d] = x.shape
    
    xc = np.copy(x)
    yc = np.copy(y)
    
    nb = math.floor(pb*n)
    
    if nb > n:
        print('Choose less data to corrupt.')
        return
    
    badsamp = np.array(rnd.sample((range(n)), nb))
    xc[badsamp] = (1/(nb*alpha))*np.dot(np.transpose(yc[badsamp]), xc[badsamp])
    yc[badsamp] = -beta
    
    return [xc, yc]

# Gradient Centered Removal of Points

In [ ]:
def gradRemoval(x, y, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, maxit, p, gam, maxIters, eps, x0):
    '''
    Gradient Centered Removal of Outliers (Regression)
    
    INPUTS:
    -------
    x = feature matrix
    y = target vector
    FactorizedRegularizedCG = ridge regression solver
    BlackBoxA = function for computing LHS in RR problem
    regGrad = function for computing gradient of objective
    lam = regularization parameter (unscaled)
    maxit = max iterations for SEVER
    p = fraction of points to throw out through filter
    gam = regularization parameter (scaled)
    maxIters = maximum iterations for solver
    eps = error tolerance for solver
    x0 = initial guess for w
    
    OUTPUTS:
    -------
    w = optimal parameter after outliers have been filtered
    it = iterations of SEVER
    t_s = run time
    '''
    
    tstart = time.time()
    
    cs = x.shape[0]
    s = np.array(range(cs))
    x1 = x
    y1 = y
    it = 0
    
    # Find optimal solution using all data
    [w, iters, t] = FactorizedRegularizedCG(BlackBoxA, x, y, gam, maxIters, eps, x0)
    
    while it < maxit:
        
        # Extract new indices
        x = x1[s, :]
        y = y1[s]
        
        # Find optimal solution from set of s points
        [w, iters, t] = FactorizedRegularizedCG(BlackBoxA, x, y, gam, maxIters, eps, w)
        
        # Compute the gradient at s points
        gradmat = regGrad(x, y, w, lam)
        
        # Form G matrix which is gradient at each point minus the average
        # G has s rows (number of used indices) and d columns (dimension of data)
        # Thus each row of G is grad_i-ave_grad
        gradhat = (1/cs)*np.sum(gradmat, axis=1)
        g = gradmat - np.array([gradhat,]*cs).transpose()
        g = g.transpose()
        
        # Compute outlier scores
        grem = la.norm(g, axis=1)
        grem = grem.flatten()
        
        # Compute fraction of outliers needed to discard
        n = len(grem)
        frac = math.floor(p*n)
    
        # Sort tao smallest to largest value
        # This returns indices
        grem = np.argsort(grem)
    
        # Discard high values
        ind = grem[0:(n - frac)]
    
        # Take indices from previous candidate set
        s = s[ind]
        
        cs = len(s)
        
        if cs == 0:
            print('You have chosen too many outliers to remove at each iteration.')
            break
        
        # Increase iteration count
        it = it + 1
    
    # Report run time of SEVER in milliseconds
    tend = time.time()
    t_s = (tend - tstart)*1000
        
    return [w, it, t_s]

# Proposed SEVER (theory supported)

In [ ]:
def SEVERfilt(s, tao, sigma, c):
    '''
    Proposed filter function for sever.
    
    INPUTS:
    -------
    s = set of indices
    tao = vector of outlier scores
    sigma = variance parameter
    c = constant > 1
    
    OUTPUTS:
    -------
    s = indices that meet filter criteria
    '''
    
    test = np.sum(tao)
    
    # Check if outlier scores meet variance criteria
    if test <= sigma*c:
        s = s
    
    else:
        T = np.random.uniform(low=0, high=np.max(tao))
        ind = np.where(tao < T)[0]
    
        # Take indices from previous candidate set
        s = s[ind]
    
    return s

In [ ]:
def severRegTheory(x, y, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, filt, sigma, c, gam, maxIters, eps, x0):
    '''
    Theoretical SEVER Algorithm for Sochastic Optimization (Regression)
    
    INPUTS:
    -------
    x = feature matrix
    y = target vector
    FactorizedRegularizedCG = ridge regression solver
    BlackBoxA = function for computing LHS in RR problem
    regGrad = function for computing gradient of objective
    lam = regularization paramter (unscaled)
    filt = filtration algorithm to select outliers
    sigma = variance paramter
    c = constant > 1
    gam = regularization parameter (scaled)
    maxIters = maximum iterations for solver
    eps = error tolerance for solver
    x0 = initial guess for w
    
    OUTPUTS:
    -------
    w = optimal parameter after outliers have been filtered
    it = iterations of SEVER
    t_s = run time
    '''
    
    tstart = time.time()
    
    cs = x.shape[0]
    s = np.array(range(cs))
    s1 = np.array([])
    x1 = x
    y1 = y
    it = 0
    
    # Find optimal solution using all data
    [w, iters, t] = FactorizedRegularizedCG(BlackBoxA, x, y, gam, maxIters, eps, x0)
    
    while cs != len(s1):
        
        # Extract new indices
        x = x1[s, :]
        y = y1[s]
        
        # Find optimal solution from set of s points
        [w, iters, t] = FactorizedRegularizedCG(BlackBoxA, x, y, gam, maxIters, eps, w)
        
        # Compute the gradient at s points
        gradmat = regGrad(x, y, w, lam)
        
        # Form G matrix which is gradient at each point minus the average
        # G has s rows (number of used indices) and d columns (dimension of data)
        # Thus each row of G is grad_i-ave_grad
        gradhat = (1/cs)*np.sum(gradmat, axis=1)
        g = gradmat - np.array([gradhat,]*cs).transpose()
        g = g.transpose()
        
        # Get largest right singluar vector in the 2-norm
        [u, sig, v] = np.linalg.svd(g, full_matrices=True)
        vh = v[0, :]
        
        # Compute outlier scores
        tao = np.square(np.dot(g, vh))
        s1 = s
        s = filt(s1, tao, sigma, c)
        cs = len(s)
        
        if cs == len(s1):
            if np.sum(s == s1) == cs:
                break
        
        # Increase iteration count
        it = it + 1
    
    # Report run time of SEVER in milliseconds
    tend = time.time()
    t_s = (tend - tstart)*1000
        
    return [w, it, t_s]

# Practical SEVER (paper used)

In [ ]:
def filt(s, tao, p):
    '''
    Practical filter function for sever.
    
    INPUTS:
    -------
    s = set of indices
    tao = vector of outlier scores
    p = fraction of points to filter
    
    OUTPUTS:
    -------
    s = indices that meet filter criteria
    '''

    n = len(tao)
    
    # Compute fraction of outliers needed to discard
    frac = math.floor(p*n)
    
    # Sort tao smallest to largest value
    # This returns indices
    tao = np.argsort(tao)
    
    # Discard high values
    ind = tao[0:(n - frac)]
    
    # Take indices from previous candidate set
    s = s[ind]
    
    return s

In [ ]:
def severReg(x, y, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, filt, maxit, p, gam, maxIters, eps, x0):
    '''
    Practical SEVER Algorithm for Sochastic Optimization (Regression)
    
    INPUTS:
    -------
    x = feature matrix
    y = target vector
    FactorizedRegularizedCG = ridge regression solver
    BlackBoxA = function for computing LHS in RR problem
    regGrad = function for computing gradient of objective
    lam = regularization paramter (unscaled)
    filt = filtration algorithm to select outliers
    maxit = max iterations for SEVER
    p = fraction of points to throw out through filter
    gam = regularization parameter (scaled)
    maxIters = maximum iterations for solver
    eps = error tolerance for solver
    x0 = initial guess for w
    
    OUTPUTS:
    -------
    w = optimal parameter after outliers have been filtered
    it = iterations of SEVER
    t_s = run time
    '''
    
    tstart = time.time()
    
    cs = x.shape[0]
    s = np.array(range(cs))
    x1 = x
    y1 = y
    it = 0
    
    # Find optimal solution using all data
    [w, iters, t] = FactorizedRegularizedCG(BlackBoxA, x, y, gam, maxIters, eps, x0)
    
    while it < maxit:
        
        # Extract new indices
        x = x1[s, :]
        y = y1[s]
        
        # Find optimal solution from set of s points
        [w, iters, t] = FactorizedRegularizedCG(BlackBoxA, x, y, gam, maxIters, eps, w)
        
        # Compute the gradient at s points
        gradmat = regGrad(x, y, w, lam)
        
        # Form G matrix which is gradient at each point minus the average
        # G has s rows (number of used indices) and d columns (dimension of data)
        # Thus each row of G is grad_i-ave_grad
        gradhat = (1/cs)*np.sum(gradmat, axis=1)
        g = gradmat - np.array([gradhat,]*cs).transpose()
        g = g.transpose()
        
        # Get largest right singluar vector in the 2-norm
        [u, sig, v] = np.linalg.svd(g, full_matrices=True)
        vh = v[0, :]
        
        # Compute outlier scores
        tao = np.square(np.dot(g, vh))
        s = filt(s, tao, p)
        cs = len(s)
        
        if cs == 0:
            print('You have chosen too many outliers to remove at each iteration.')
            break
        
        # Increase iteration count
        it = it + 1
    
    # Report run time of SEVER in milliseconds
    tend = time.time()
    t_s = (tend - tstart)*1000
        
    return [w, it, t_s]

# Numerical Experiments

### Synthetic Data

In [ ]:
# Set up to compare SEVER to CG method

# Generate synthetic data as described in the paper
n = 5000; t = 100; l = 500; w_true = np.random.uniform(low=0, high=2*math.pi, size=l)

# Add details about attack data
nb = math.floor(n*0.02)                         # number of data that is "corrupted"
alp = np.random.uniform(low=0.001, high=100)    # hyperparameter to increase effectiveness of attack
bet = np.random.uniform(low=0.001, high=100)    # hyperparameter to increase effectiveness of attack

[x_train_reg, y_train_reg, x_test_reg, y_test_reg] = genRan(n, t, l, w_true, tog=0, nbad=nb, alpha=alp, beta=bet)

# Specifics for CG methods
lam = 0.001; gam = n*lam; maxIters = 50; eps = 10**(-12); x0 = np.zeros((500,))
maxit = 4                                       # total number of iterations for practical SEVER
sigma = 1000; c = 10                            # variance parameters for theoretical SEVER

In [ ]:
# Compare SEVER on test data to other methods:

compnorm = la.norm(y_test_reg)
# Find optimal w
[w_sevtheory, sevtheory_it, t_sevtheory] = severRegTheory(x_train_reg, y_train_reg, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, SEVERfilt, sigma, c, gam, maxIters, eps, x0)
[w_sever, sever_it, t_sever] = severReg(x_train_reg, y_train_reg, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, filt, maxit, (0.02)/2, gam, maxIters, eps, x0)
[w_grad, grad_it, t_grad] = gradRemoval(x_train_reg, y_train_reg, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, maxit, (0.02)/2, gam, maxIters, eps, x0)
[w_cg, cg_it, t_cg] = FactorizedRegularizedCG(BlackBoxA, x_train_reg, y_train_reg, gam, maxIters*maxit, eps, x0)
print('Practical SEVER runtime in ms: ', round(t_sever, 2), '\nTheoretical SEVER runtime in ms: ', round(t_sevtheory, 2), '\nGradient Centered Removal runtime in ms: ', round(t_grad, 2), '\nCG runtime in ms: ', round(t_cg, 2))

# Use w^* on uncorrupted testing data
sever_error = (1/t)*la.norm(np.dot(x_test_reg, w_sever) - y_test_reg)/compnorm
severtheory_error = (1/t)*la.norm(np.dot(x_test_reg, w_sevtheory) - y_test_reg)/compnorm
grad_error = (1/t)*la.norm(np.dot(x_test_reg, w_grad) - y_test_reg)/compnorm
cg_error = (1/t)*la.norm(np.dot(x_test_reg, w_cg) - y_test_reg)/compnorm
print('Relative Test Error for Practical SEVER: ', round(sever_error, 6),'\nRelative Test Error for Theoretical SEVER: ', round(severtheory_error, 6),'\nRelative Test Error for Gradient Centered l-2 Removal: ', round(grad_error, 6), '\nRelative Test Error for Regular Ridge Regression with CG: ', round(cg_error, 6))

In [ ]:
# Compare SEVER over a range of attack data

compnorm = la.norm(y_test_reg)

# Create vector to test
nbvec = np.array([0, int(0.01*n), int(0.02*n), int(0.03*n), int(0.04*n), int(0.05*n), int(0.06*n), int(0.07*n), int(0.08*n), int(0.09*n), int(0.1*n)])

Err_sever = np.array([]); Err_severtheory = np.array([]); Err_cg = np.array([]); Err_grad = np.array([])
time_sever = np.array([]); time_severtheory = np.array([]); time_cg = np.array([]); time_grad = np.array([])

for i in range(len(nbvec)):
    # Generate data
    [x_train_reg, y_train_reg, x_test_reg, y_test_reg] = genRan(n, t, l, w_true, tog=0, nbad=nbvec[i], alpha=alp, beta=bet)
    
    # Run practical SEVER
    [w_sever, sever_it, t_sever] = severReg(x_train_reg, y_train_reg, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, filt, maxit, (nbvec[i])/(n*2), gam, maxIters, eps, x0)
    
    # Run theoretical SEVER
    [w_sevtheory, sevtheory_it, t_sevtheory] = severRegTheory(x_train_reg, y_train_reg, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, SEVERfilt, sigma, c, gam, maxIters, eps, x0)
    
    # Run gradient centered removal
    [w_grad, grad_it, t_grad] = gradRemoval(x_train_reg, y_train_reg, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, maxit, (nbvec[i])/(n*2), gam, maxIters, eps, x0)
    
    # Compute Ridge Regression solution
    [w_cg, it_cg, t_cg] = FactorizedRegularizedCG(BlackBoxA, x_train_reg, y_train_reg, gam, maxIters*maxit, eps, x0)
    
    # Compute testing errors
    sever_error = (1/t)*la.norm(np.dot(x_test_reg, w_sever) - y_test_reg)/compnorm
    severtheory_error = (1/t)*la.norm(np.dot(x_test_reg, w_sevtheory) - y_test_reg)/compnorm
    grad_error = (1/t)*la.norm(np.dot(x_test_reg, w_grad) - y_test_reg)/compnorm
    cg_error = (1/t)*la.norm(np.dot(x_test_reg, w_cg) - y_test_reg)/compnorm
    
    # Record testing errors
    Err_sever = np.append(Err_sever, sever_error)
    Err_severtheory = np.append(Err_severtheory, severtheory_error)
    Err_grad = np.append(Err_grad, grad_error)
    Err_cg = np.append(Err_cg, cg_error)
    
    # Record time
    time_sever = np.append(time_sever, t_sever)
    time_severtheory = np.append(time_severtheory, t_sevtheory)
    time_grad = np.append(time_grad, t_grad)
    time_cg = np.append(time_cg, t_cg)

# Ensure dimensions are appropriate for plotting
Err_sever = Err_sever.flatten()
Err_severtheory = Err_severtheory.flatten()
Err_grad = Err_grad.flatten()
Err_cg = Err_cg.flatten()

time_sever = time_sever.flatten()
time_severtheory = time_severtheory.flatten()
time_grad = time_grad.flatten()
time_cg = time_cg.flatten()

In [ ]:
# Plots of SEVER test error over a range of attacks

xax = np.linspace(nbvec[0], nbvec[len(nbvec)-1], len(nbvec), endpoint=True); xax = (xax/n)
plt.plot(xax, Err_sever, 'b-', label='PracSEVER')
plt.plot(xax, Err_cg, 'r-', label='CG')
plt.plot(xax, Err_severtheory, 'k-', label='TheorySEVER')
plt.plot(xax, Err_grad, 'g-', label='GradCenter')
plt.title('Synthetic Data Error')
plt.ylabel('RMSE/$||y||_2$')
plt.xlabel('Percent of Corrupted Training Points')
plt.legend()
# plt.savefig('regressionSEVER.png')

In [ ]:
# Compare runtime of different algorithms

plt.plot(xax, time_sever, 'b-', label='PracSEVER')
plt.plot(xax, time_cg, 'r-', label='CG')
plt.plot(xax, time_severtheory, 'k-', label='TheorySEVER')
plt.plot(xax, time_grad, 'g-', label='GradCenter')
plt.title('Algorithm Run Time')
plt.ylabel('Time (ms)')
plt.xlabel('Percent of Corrupted Training Points')
plt.legend()
# plt.savefig('regressionSEVERtime.png')

# Comparison of SEVER Across Corruption Amounts
---
- Synthetic data has percent outliers $\varepsilon\in\{0.01,0.02,\dots,0.1\}$
- For `PracSEVER` and `GradCenter` we remove $p=\frac{\varepsilon\cdot n}{2}$ points
<table><tr><td><img src='regressionSEVER.png'></td><td><img src='regressionSEVERtime.png'></td></tr></table>


### Song Prediction Data

In [ ]:
# Year prediction data

# Choose number of training and testing data
numtrain = 5000
numtest = 200

# Training data
df_train = pd.read_csv('YearPredictionMSD.txt', header=None, nrows=numtrain)
df_train = pd.DataFrame(df_train)
y_train = df_train[0]
ytrain = np.array(y_train)
X_train = df_train.drop(columns=[0])
xtrain = np.array(X_train)

# Testing data
df_test = pd.read_csv('YearPredictionMSD.txt', header=None, skiprows=463715, nrows=numtest)
df_test = pd.DataFrame(df_test)
y_test = df_test[0]
ytest = np.array(y_test)
X_test = df_test.drop(columns=[0])
xtest = np.array(X_test)

In [ ]:
# Corrupt training data
pcor = 0.1
[xtrain_c, ytrain_c] = dataAttack(xtrain, ytrain, pb=pcor, alpha=-np.random.uniform(0.001, 50), beta=np.random.uniform(1900, 2100))


In [ ]:
maxiter = 3; x0 = np.zeros((90,)); eps = 10**(-5); cgiters = 100; lam = 0.01; gamma = numtrain*lam

# Preprocess data
xtrainnew = xtrain - np.mean(xtrain, axis=0)
xtestnew = xtest - np.mean(xtest, axis=0)
ytrainnew = ytrain - np.mean(ytrain)
ytestnew = ytest - np.mean(ytest)
normcomp2 = la.norm(ytestnew)
xtrainnewc = xtrain_c - np.mean(xtrain_c, axis=0)
ytrainnewc = ytrain_c - np.mean(ytrain_c)

# Practical SEVER without corruptions
[w_yp_p, it_yp_p, t_yp_p] = severReg(xtrainnew, ytrainnew, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, filt, maxiter, (pcor/2), gamma, cgiters, eps, x0)
yp_error_p = (1/numtest)*la.norm(np.dot(xtestnew, w_yp_p) - ytestnew)/normcomp2
print('SEVER RMSE without corruptions: ',round(yp_error_p,6))

# Regular Conjugate Gradient without corruptions
[w_yp_cg, it_yp_cg, t_yp_cg] = FactorizedRegularizedCG(BlackBoxA, xtrainnew, ytrainnew, gamma, maxiter*cgiters, eps, x0)
yp_error_cg = (1/numtest)*la.norm(np.dot(xtestnew, w_yp_cg) - ytestnew)/normcomp2
print('CG RMSE without corruptions: ',round(yp_error_cg,6))

maxiter = 2;
# Practical SEVER with corruptions
[w_yp_p_c, it_yp_p_c, t_yp_p_c] = severReg(xtrainnewc, ytrainnewc, FactorizedRegularizedCG, BlackBoxA, regGrad, lam, filt, maxiter, (pcor)/2, gamma, cgiters, eps, x0)
yp_error_p_c = (1/numtest)*la.norm(np.dot(xtestnew, w_yp_p_c) - ytestnew)/normcomp2
print('SEVER RMSE with corruptions: ',round(yp_error_p_c,6))

# Regular Conjugate Gradient with corruptions
[w_yp_cg_c, it_yp_cg_c, t_yp_cg_c] = FactorizedRegularizedCG(BlackBoxA, xtrainnewc, ytrainnewc, gamma, maxiter*cgiters, eps, x0)
yp_error_cg_c = (1/numtest)*la.norm(np.dot(xtestnew, w_yp_cg_c) - ytestnew)/normcomp2
print('CG RMSE with corruptions: ',round(yp_error_cg_c,6))


# SEVER Comparison on Real Data
---
- Use "Year Prediction Data"
- Test on original data and on corrupted data
- Center target vector and data
- Use `PracSEVER` only

|Method|Corrupt?|RMSE/$||y||_2$|
|---|---|---|
|`PracSEVER`|No|0.004076|
|`CG`|No|0.004065|
|`PracSEVER`|Yes|0.006808|
|`CG`|Yes|0.006731|

# SEVER + Clustering?
---
- Does this type of filtering extend to other machine learning methods?
- Use `Iris` dataset 
- Use K-means with cluster set at 3
- Use the **practical** filtering method

# Numerical Results
---
- Toy example: just use the `Sepal Width` and `Pedal Width` features
<table><tr><td><img src='Iris2d.png'></td><td>Initial Training Accuracy: 4%<br>Initial Testing Accuracy: 4%<br>Training Accuracy after SEVER: 37%<br>Testing Accuracy after SEVER: 44%</td></tr></table>

# Using the Entire Iris Dataset
---
- Run `SEVER` 50 times
- Filter out 2% of the data every time

|`SEVER`?|Training Accuracy|Testing Accuracy|
|---|---|---|
|No|87%|90%|
|Yes|85%|92%|

Note: runtime is about 1 second

## Clustering Code

In [ ]:
from sklearn.cluster import KMeans as km
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris = datasets.load_iris()
xiris = iris.data[:, (1,3)]
xirisfull = iris.data
yiris = iris.target

# Separate into training and testing data
xtrain, xtest, ytrain, ytest = train_test_split(xiris, yiris, test_size=0.33, random_state=42)
xtrainfull, xtestfull, ytrainfull, ytestfull = train_test_split(xirisfull, yiris, test_size=0.33, random_state=42)

In [ ]:
def SEVERCluster(x, y, xt, yt, kmnum, filt, p, maxit):
    '''
    SEVER for Clustering
    
    INPUTS:
    -------
    x = training features
    y = training targets
    xt = testing features
    yt = testing targets
    kmnum = number of clusters
    filt = filtering algorithm
    p = percent of points thrown out each time
    maxit = number of iterations of SEVER
    
    OUTPUTS:
    -------
    ftrain_acc = final training accuracy
    ftest_acc = final testing accuracy
    t_s = run time of algorithm
    xf = final or 'reduced' feature matrix
    fcentroids = final centroids
    '''
    
    tstart = time.time()
    
    cs = x.shape[0]
    s = np.array(range(cs))
    x1 = np.copy(x)
    y1 = np.copy(y)
    
    # Precompute for efficiency
    gradx = np.sum(x, axis=0)
    
    # Test on unaltered data
    kmeans = km(n_clusters=kmnum, random_state=0).fit(x)
    print('Initial training accuracy: ',sum(kmeans.labels_==y)/len(y))
    print('Initial testing accuracy: ',sum(kmeans.predict(xt)==yt)/len(yt))
    
    it = 0
    
    while it <= maxit:
        
        # Extract new indices
        x = x1[s, :]
        y = y1[s]
        
        # Cluster on x[s]
        kmeans = km(n_clusters=kmnum, random_state=0).fit(x)
        
        centroids = kmeans.cluster_centers_
        
        # Find which center is closest to the data point
        norms = np.zeros((x.shape[0], kmnum))
        
        centroids = kmeans.cluster_centers_
        
        for i in range(kmnum):
            norms[:,i] = la.norm(x - centroids[i,:], axis=1)
            
        mins = np.argmin(norms, axis=1)
        
        gradmat = x - centroids[mins]
        
        # Form G matrix which is gradient at each point minus the average
        # G has s rows (number of used indices) and d columns (dimension of data)
        # Thus each row of G is grad_i-ave_grad
        gradhat = (1/cs)*np.sum(gradmat, axis=0)
        g = gradmat - np.array([gradhat,]*cs)
        g = g
        
        # Get largest right singluar vector in the 2-norm
        [u, sig, v] = np.linalg.svd(g, full_matrices=True)
        vh = v[0, :]
        
        # Compute outlier scores
        tao = np.square(np.dot(g, vh))
        s = filt(s, tao, p)
        cs = len(s)
        
        if cs == 0:
            print('You have chosen too many outliers to remove at each iteration.')
            break
         
        it = it + 1
        
    # Save reduced x
    xf = np.copy(x)
    
    # Report run time of SEVER in milliseconds
    tend = time.time()
    t_s = (tend - tstart)*1000
    
    # Test on testing data
    ftrain_acc = sum(kmeans.labels_==y)/len(y)
    ftest_acc = sum(kmeans.predict(xt)==yt)/len(yt)
    fcentroids = kmeans.cluster_centers_
    
    return [ftrain_acc, ftest_acc, t_s, xf, fcentroids]

In [ ]:
# Use only part of the data

[acctrain, acctest, tim, xtrain_red, centers] = SEVERCluster(xtrain, ytrain, xtest, ytest, 3, filt, 0.02, 10)
#print(acctrain)
#print(acctest)
plt.scatter(xtrain[:,0], xtrain[:,1], facecolors='none', color='b', linewidth=0.5, label='BeforeSEVER')
plt.scatter(xtrain_red[:,0], xtrain_red[:,1], marker="+", color='r', label='AfterSEVER')
plt.scatter(centers[:,0], centers[:,1], color='k', marker="*", label='Centers')
plt.title('Iris Data Points before and after SEVER')
plt.ylabel('Pedal Width')
plt.xlabel('Sepal Width')
plt.legend()
#plt.savefig('Iris2d.png')
plt.show()

In [ ]:
# Use all of the data

[acctrain, acctest, tim, xtrain_red, centers] = SEVERCluster(xtrainfull, ytrainfull, xtestfull, ytestfull, 3, filt, 0.02, 50)
print(acctrain)
print(acctest)

# Conclusion
---
|Pros|Cons|
|---|---|
|Relatively easily implemented|Potentially slow runtime|
|*Usually* gets better results|Potentially at the cost of tuning hyperparamters|
|Generalizes to other problem settings|Data cannot be coupled|
|Has theoretical backing|Some issues in the proofs|

# Thank You